In [119]:
import onnxruntime as rt
import numpy as np

onnx_model_v1 = "../data/models/birdnet-onnx-v1.onnx"

test = np.random.randn(10, 144_000).astype(np.float32)

sess = rt.InferenceSession(onnx_model_v1)
y = sess.run(None, {"inputs:0": test})
[res.shape for res in y]

[(320,), (3337,)]

In [25]:
import tensorflow as tf

opt_graph_v1 = "../data/models/birdnet-opt-v1.pb"

# load optimized graph
with tf.io.gfile.GFile(opt_graph_v1, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# y = eval_graph(graph_def, test)
with tf.compat.v1.Session() as sess:
    y = tf.graph_util.import_graph_def(
        graph_def,
        input_map={"inputs:0": tf.convert_to_tensor(test)},
        return_elements=["unknown_242:0", "unknown_244:0"],
    )
y

[<tf.Tensor 'import_8/unknown_242:0' shape=(320,) dtype=float32>,
 <tf.Tensor 'import_8/unknown_244:0' shape=(3337,) dtype=float32>]

We're getting the wrong size for the output model, we only get the first element...

In [87]:
tf_bin_path = "../data/bin/linux/"
model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"
new_model_path_v2 = "../data/models/birdnet-multi-v2"
frozen_graph_v2 = "../data/models/birdnet-frozen-v2.pb"
opt_graph_v2 = "../data/models/birdnet-opt-v2.pb"
onnx_model_v2 = "../data/models/birdnet-onnx-v2.onnx"

In [27]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

m = tf.saved_model.load(model_path)

In [28]:
m.model(tf.convert_to_tensor(test))

<tf.Tensor: shape=(5, 3337), dtype=float32, numpy=
array([[ -9.155447 ,  -8.683059 ,  -7.0943418, ...,  -9.102758 ,
         -8.269766 ,  -9.981155 ],
       [ -9.148126 ,  -8.591087 ,  -7.520302 , ...,  -9.292491 ,
         -7.991625 , -10.144447 ],
       [ -9.321141 ,  -8.479657 ,  -7.8567896, ...,  -9.924754 ,
         -8.134635 , -10.7452345],
       [ -9.379911 ,  -8.625297 ,  -7.3190327, ...,  -9.123431 ,
         -8.381328 , -10.365876 ],
       [ -9.45595  ,  -8.965809 ,  -7.686916 , ...,  -9.740916 ,
         -8.15754  , -10.688493 ]], dtype=float32)>

In [32]:
[x for x in dir(m) if not x.startswith("_")]

['basic',
 'graph_debug_info',
 'model',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version']

In [33]:
m.basic

In [34]:
m.signatures["basic"]

<ConcreteFunction signature_wrapper(*, inputs) at 0x7FDBCCDB1B10>

In [95]:
signatures = {
    "basic": m.signatures["basic"],
    "array_input": m.model.__call__.get_concrete_function(
        tf.TensorSpec([10, 144_000], dtype=tf.float32)
    ),
}
print(signatures["basic"].pretty_printed_signature())
print(signatures["array_input"].pretty_printed_signature())

tf.saved_model.save(m, new_model_path_v2, signatures=signatures)

signature_wrapper(*, inputs)
  Args:
    inputs: float32 Tensor, shape=(None, 144000)
  Returns:
    {'scores': <1>}
      <1>: float32 Tensor, shape=(None, 3337)
restored_function_body(inputs, training=False, mask=None)
  Args:
    inputs: float32 Tensor, shape=(10, 144000)
  Returns:
    float32 Tensor, shape=(10, 3337)
INFO:tensorflow:Assets written to: ../data/models/birdnet-multi-v2/assets


INFO:tensorflow:Assets written to: ../data/models/birdnet-multi-v2/assets


In [96]:
# reload new model
m2 = tf.saved_model.load(new_model_path_v2)
m2.signatures

_SignatureMap({'basic': <ConcreteFunction signature_wrapper(*, inputs) at 0x7FDB310D72B0>, 'array_input': <ConcreteFunction signature_wrapper(*, inputs) at 0x7FDB30B9FC70>})

In [107]:
frozen_func = convert_variables_to_constants_v2(m2.signatures["array_input"])
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="./",
    name=frozen_graph_v2,
    as_text=False,
)

2023-03-19 00:16:07.101931: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-03-19 00:16:07.102041: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


'./../data/models/birdnet-frozen-v2.pb'

In [123]:
! {tf_bin_path}/transform_graph \
    --in_graph={frozen_graph_v2} \
    --out_graph={opt_graph_v2} \
    --inputs='inputs:0' \
    --transforms=' \
        remove_nodes(op=CheckNumerics) \
        fold_old_batch_norms \
    '

2023-03-19 00:19:22.827850: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 00:19:22.877127: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2023-03-19 00:19:22.945844: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying fold_old_batch_norms


In [124]:
! {tf_bin_path}/summarize_graph \
    --in_graph={opt_graph_v2} \
    --print_structure=true

2023-03-19 00:19:25.606469: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Found 1 possible inputs: (name=inputs, type=float(1), shape=[10,144000]) 
No variables spotted.
Found 1 possible outputs: (name=Identity, op=Identity) 
Found 5488973 (5.49M) const parameters, 0 (0) variable parameters, and 2238 control_edges
Op types used: 1853 Identity, 363 Const, 313 NoOp, 53 Conv2D, 49 Mul, 45 FusedBatchNormV3, 43 Sigmoid, 34 IdentityN, 17 AddV2, 15 Pack, 14 Reshape, 12 StridedSlice, 10 Shape, 10 Mean, 9 DepthwiseConv2dNative, 7 Sub, 4 FloorDiv, 4 ConcatV2, 4 Cast, 4 Range, 3 Relu, 3 RealDiv, 2 Pow, 2 Add, 1 RFFT, 1 SplitV, 1 Transpose, 1 Placeholder, 1 Min, 1 Maximum, 1 MaxPool, 1 Max, 1 MatMul, 1 GatherV2, 1 FloorMod, 1 Fill, 1 ExpandDims, 1 Exp, 1 Cos, 1 AvgPool
To use with 

- https://stackoverflow.com/questions/63618671/how-to-run-inference-using-tensorflow-2-2-pb-file

In [128]:
! python -m tf2onnx.convert \
  --input {opt_graph_v2} \
  --inputs inputs:0 \
  --outputs unknown_242:0,unknown_244:0 \
  --output {onnx_model_v2}

2023-03-19 00:20:23.827826: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 00:20:23.935124: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-19 00:20:24.427100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-19 00:20:24.427166: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [129]:
with tf.io.gfile.GFile(opt_graph_v2, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# y = eval_graph(graph_def, test)
with tf.compat.v1.Session() as sess:
    y = tf.graph_util.import_graph_def(
        graph_def,
        input_map={"inputs:0": tf.convert_to_tensor(test)},
        return_elements=["unknown_242:0"],
    )
y, test.shape

([<tf.Tensor 'import_31/unknown_242:0' shape=(320,) dtype=float32>],
 (10, 144000))

In [130]:
sess = rt.InferenceSession(onnx_model_v2)
y = sess.run(None, {"inputs:0": test})
[res.shape for res in y]

[(320,), (3337,)]